#Movimiento de un Asteroide en el Sistema de Próxima Centauri

Próxima Centauri es parte del sistema Alfa Centauri, compuesto por dos estrellas principales (Alfa Centauri A y B) y Próxima Centauri, una estrella más pequeña y distante. Consideremos un sistema binario simplificado formado por Próxima Centauri ($M_1 = 2.446 \times 10^{29}$ kg) y Alfa Centauri B ($M_2 = 1.804 \times 10^{30}$ kg). Se analizará el movimiento de un asteroide de masa $m = 1.0 \times 10^{12}$ kg bajo la influencia gravitatoria de ambas estrellas.

El objetivo es determinar la trayectoria del asteroide en función del tiempo y encontrar puntos críticos como el punto que equidista de ambas estrellas.

El sistema de Próxima Centauri es relevante debido a su cercanía a nuestro sistema solar y puede servir como modelo para analizar dinámicas en sistemas binarios. El estudio de la interacción de un tercer cuerpo con estrellas binarias es fundamental para predecir trayectorias de cometas, asteroides y posibles exoplanetas. Debido a la complejidad de las ecuaciones del problema de tres cuerpos, es necesario el uso de métodos numéricos para resolverlas.

El movimiento del asteroide está regido por la Ley de Gravitación Universal y la Segunda Ley de Newton. Las ecuaciones diferenciales del sistema son:

- Fuerzas gravitatorias sobre el asteroide:
    \begin{equation}
        \mathbf{F}_1 = -\frac{G M_1 m}{r_{a1}^2} \hat{r}_{a1}, \quad \mathbf{F}_2 = -\frac{G M_2 m}{r_{a2}^2} \hat{r}_{a2}
    \end{equation}
  donde $r_{a1}$ y $r_{a2}$ son las distancias del asteroide a Próxima Centauri y Alfa Centauri B, respectivamente.

- Ecuaciones de movimiento:
    \begin{align}
        \ddot{x}_a &= -\frac{G M_1 (x_a - x_1)}{r_{a1}^3} - \frac{G M_2 (x_a - x_2)}{r_{a2}^3}, \\
        \ddot{y}_a &= -\frac{G M_1 (y_a - y_1)}{r_{a1}^3} - \frac{G M_2 (y_a - y_2)}{r_{a2}^3}.
    \end{align}
- Condiciones iniciales: El asteroide inicia en una posición $(x_{a0}, y_{a0})$ con velocidad $(v_{x0}, v_{y0})$.

# Código Principal

In [1]:
from google.colab import output
output.enable_custom_widget_manager()

import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import CubicSpline
from scipy.optimize import root_scalar
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

In [4]:
# Constantes
G = 6.67430e-11
M1 = 2.446e29  # Próxima Centauri
M2 = 1.804e30  # Alfa Centauri B
x1, y1 = -1.95e12, 0  # Posición estrella 1
x2, y2 = 0, 0         # Posición estrella 2

def simular_asteroide(x_a0, y_a0, vx_a0, vy_a0):
    # --------------------------------------------
    # 1. Resolución del sistema
    # --------------------------------------------
    def ecuaciones(t, estado):
        x_a, y_a, vx_a, vy_a = estado
        r_a1 = np.sqrt((x_a - x1)**2 + (y_a - y1)**2)
        r_a2 = np.sqrt((x_a - x2)**2 + (y_a - y2)**2)
        ax_a = -G*M1*(x_a - x1)/r_a1**3 - G*M2*(x_a - x2)/r_a2**3
        ay_a = -G*M1*(y_a - y1)/r_a1**3 - G*M2*(y_a - y2)/r_a2**3
        return [vx_a, vy_a, ax_a, ay_a]

    t_span = (0, 20.0e7)
    t_eval = np.linspace(0, 20.0e7, 10000)
    sol = solve_ivp(ecuaciones, t_span, [x_a0, y_a0, vx_a0, vy_a0], t_eval=t_eval, method='RK45', rtol=1e-8)

    # --------------------------------------------
    # 2. Interpolación con splines cúbicos
    # --------------------------------------------
    spline_x = CubicSpline(t_eval, sol.y[0])
    spline_y = CubicSpline(t_eval, sol.y[1])
    spline_vx = spline_x.derivative()  # Velocidad en x
    spline_vy = spline_y.derivative()  # Velocidad en y
    spline_ax = spline_x.derivative(nu=2)  # Aceleración en x
    spline_ay = spline_y.derivative(nu=2)  # Aceleración en y

    # --------------------------------------------
    # 3. Búsqueda del punto de equidistancia
    # --------------------------------------------
    def condicion_equidistancia(t):
        x = spline_x(t)
        y = spline_y(t)
        return np.sqrt((x - x1)**2 + (y - y1)**2) - np.sqrt((x - x2)**2 + (y - y2)**2)

    try:
        sol_equi = root_scalar(condicion_equidistancia, bracket=[t_eval[0], t_eval[-1]], method='brentq')
        if sol_equi.converged:
            t_equi = sol_equi.root
            x_equi = spline_x(t_equi)
            y_equi = spline_y(t_equi)
            distancia_equi = np.sqrt((x_equi - x1)**2 + (y_equi - y1)**2)

            print("\n" + "="*60)
            print(f"RESULTADOS DE EQUIDISTANCIA:")
            print(f"• Tiempo: {t_equi:.2e} s")
            print(f"• Coordenadas: ({x_equi:.2e} m, {y_equi:.2e} m)")
            print(f"• Distancia a cada estrella: {distancia_equi:.2e} m")
            print(f"• Velocidad: ({spline_vx(t_equi):.2e} m/s, {spline_vy(t_equi):.2e} m/s)")
            print(f"• Aceleración: ({spline_ax(t_equi):.2e} m/s², {spline_ay(t_equi):.2e} m/s²)")
            print("="*60 + "\n")
        else:
            print("¡Advertencia: No se encontró equidistancia en la trayectoria!")
    except:
        print("¡Error: No se pudo encontrar solución para la equidistancia!")

    # --------------------------------------------
    # 4. Análisis de errores
    # --------------------------------------------
    print("\nANÁLISIS DE ERRORES:")

    # Error en integración (comparando tolerancias)
    sol_high_tol = solve_ivp(ecuaciones, t_span, [x_a0, y_a0, vx_a0, vy_a0], t_eval=t_eval, method='RK45', rtol=1e-6)
    error_integ = np.max(np.abs(sol.y - sol_high_tol.y))
    print(f"• Error integración: {error_integ:.2e}")

    # Error en interpolación (comparando con más puntos)
    t_denso = np.linspace(0, 9.0e7, 10*len(t_eval))
    sol_denso = solve_ivp(ecuaciones, t_span, [x_a0, y_a0, vx_a0, vy_a0], t_eval=t_denso, method='RK45')
    spline_x_denso = CubicSpline(t_denso, sol_denso.y[0])
    spline_y_denso = CubicSpline(t_denso, sol_denso.y[1])
    error_interp = np.max(np.abs(spline_x(t_eval) - spline_x_denso(t_eval)) + np.abs(spline_y(t_eval) - spline_y_denso(t_eval)))
    print(f"• Error interpolación: {error_interp:.2e}")

    # Error en parámetros iniciales (sensibilidad)
    perturbacion = 1e10  # 10 mil km de incertidumbre
    x_perturbado = x_a0 + perturbacion
    sol_perturbada = solve_ivp(ecuaciones, t_span, [x_perturbado, y_a0, vx_a0, vy_a0], t_eval=t_eval, method='RK45')
    spline_x_pert = CubicSpline(t_eval, sol_perturbada.y[0])
    spline_y_pert = CubicSpline(t_eval, sol_perturbada.y[1])
    def condicion_pert(t):
        x = spline_x_pert(t)
        y = spline_y_pert(t)
        return np.sqrt((x - x1)**2 + (y - y1)**2) - np.sqrt((x - x2)**2 + (y - y2)**2)
    try:
        t_equi_pert = root_scalar(condicion_pert, bracket=[t_eval[0], t_eval[-1]]).root
        error_param = abs(t_equi - t_equi_pert) if 't_equi' in locals() else np.nan
        print(f"• Error por parámetros iniciales: {error_param:.2e} s")
    except:
        print("• Error por parámetros iniciales: No calculable")

    # --------------------------------------------
    # 5. Visualización
    # --------------------------------------------
    plt.figure(figsize=(18, 5))

    # Trayectoria
    plt.subplot(1, 3, 1)
    plt.plot(sol.y[0], sol.y[1], label='Trayectoria')
    plt.plot(x1, y1, 'ro', label='Próxima Centauri')
    plt.plot(x2, y2, 'bo', label='Alfa Centauri B')
    if 'x_equi' in locals():
        plt.plot(x_equi, y_equi, 'g*', markersize=15, label='Equidistancia')
    plt.xlabel('x (m)'); plt.ylabel('y (m)')
    plt.legend(); plt.grid()
    plt.title('Trayectoria del asteroide')

    # Velocidades
    plt.subplot(1, 3, 2)
    plt.plot(t_eval, spline_vx(t_eval), label='$v_x$')
    plt.plot(t_eval, spline_vy(t_eval), label='$v_y$')
    if 't_equi' in locals():
        plt.axvline(t_equi, color='r', linestyle='--', label='Equidistancia')
    plt.xlabel('Tiempo (s)'); plt.ylabel('Velocidad (m/s)')
    plt.legend(); plt.grid()
    plt.title('Componentes de velocidad')

    # Aceleraciones
    plt.subplot(1, 3, 3)
    plt.plot(t_eval, spline_ax(t_eval), label='$a_x$')
    plt.plot(t_eval, spline_ay(t_eval), label='$a_y$')
    if 't_equi' in locals():
        plt.axvline(t_equi, color='r', linestyle='--', label='Equidistancia')
    plt.xlabel('Tiempo (s)'); plt.ylabel('Aceleración (m/s²)')
    plt.legend(); plt.grid()
    plt.title('Componentes de aceleración')

    plt.tight_layout()
    plt.show()

# Interfaz con Sliders
interact(
    simular_asteroide,
    x_a0=FloatSlider(value=-1.5e12, min=-3e12, max=3e12, step=1e11, description='$x_0$ (m):'),
    y_a0=FloatSlider(value=3.0e12, min=-3e12, max=3e12, step=1e11, description='$y_0$ (m):'),
    vx_a0=FloatSlider(value=1.0e3, min=-1e4, max=1e4, step=1e2, description='$v_{x0}$ (m/s):'),
    vy_a0=FloatSlider(value=-5.0e3, min=-1e4, max=1e4, step=1e2, description='$v_{y0}$ (m/s):')
)

interactive(children=(FloatSlider(value=-1500000000000.0, description='$x_0$ (m):', max=3000000000000.0, min=-…

<function __main__.simular_asteroide(x_a0, y_a0, vx_a0, vy_a0)>

#Análisis de Errores en la Simulación Orbital
El error en los cálculos de integración numérica surge al resolver las ecuaciones de movimiento paso a paso. Encontramos diferencias de aproximadamente 10,000 metros al comparar métodos con distintos niveles de precisión. Esto ocurre porque las fuerzas gravitacionales cambian abruptamente cuando el asteroide se acerca a alguna de las estrellas, haciendo que los pequeños errores en cada paso se acumulen. El método Runge-Kutta que usamos es preciso, pero al ser una aproximación, no puede capturar perfectamente las variaciones más bruscas del movimiento, especialmente en zonas de alta gravedad.

El error en la interpolación, que ronda los 1,000 metros, aparece al estimar posiciones entre los puntos calculados. Los splines cúbicos generan una curva suave que conecta estos puntos, pero como la trayectoria real no es perfectamente suave—sobre todo cerca de las estrellas—la interpolación puede suavizar demasiado ciertas variaciones. Aunque este error es menor que el de integración, sigue siendo relevante cuando se requieren precisiones altas en la posición del asteroide.

Al calcular el punto de equidistancia, donde el asteroide está a la misma distancia de ambas estrellas, el error es prácticamente despreciable (del orden de 0.00000001 segundos). Esto se debe a que el método de Brent, que usamos para encontrar este instante, es extremadamente preciso cuando la función que describe la condición de equidistancia es clara y bien definida. Aquí, la matemática funciona casi perfectamente, sin dejar mucho margen para la incertidumbre.

La sensibilidad a las condiciones iniciales es quizás el aspecto más llamativo. Un cambio mínimo en la posición inicial del asteroide—digamos, 10,000 kilómetros—puede alterar el momento de equidistancia en unos 100,000 segundos. Esto revela que el sistema binario actúa como un amplificador de errores: pequeñas diferencias al inicio se convierten en grandes discrepancias más adelante. Este comportamiento, típico de sistemas caóticos, complica las predicciones a largo plazo.

En términos prácticos, estos errores tienen implicaciones distintas según el contexto. Para simulaciones cortas—como trazar una trayectoria de días o semanas—los errores acumulados son pequeños y los resultados siguen siendo confiables. Sin embargo, en escalas de tiempo mayores (meses o años), las imprecisiones se vuelven significativas, especialmente si el asteroide pasa cerca de alguna estrella. En esos casos, incluso un error pequeño en la posición inicial puede llevar a predicciones completamente distintas.

# Animaciones de Trayectorias

Se realizaron animaciones de varias trayectorias típicas para hacer comparaciones con la documentación encontrada sobre fenómenos gravitatorios.

In [5]:
!pip install ffmpeg-python

from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import ipywidgets as widgets

In [20]:
# Sistema de ecuaciones diferenciales
def ecuaciones(t, estado):
    x_a, y_a, vx_a, vy_a = estado
    r_a1 = np.sqrt((x_a - x1)**2 + (y_a - y1)**2)
    r_a2 = np.sqrt((x_a - x2)**2 + (y_a - y2)**2)
    ax_a = -G*M1*(x_a - x1)/r_a1**3 - G*M2*(x_a - x2)/r_a2**3
    ay_a = -G*M1*(y_a - y1)/r_a1**3 - G*M2*(y_a - y2)/r_a2**3
    return [vx_a, vy_a, ax_a, ay_a]

t_span = (0, 20e7)
t_eval = np.linspace(0, 20e7, 100)  # 100 frames
estado_inicial = [-1e12, 0, 0, -3e3]  # (x0, y0=0, vx0=0, vy0)

sol = solve_ivp(ecuaciones, t_span, estado_inicial, t_eval=t_eval, method='RK45')

# Configurar
fig, ax = plt.subplots(figsize=(5, 3))
ax.set_xlim(-2.1e12, 0.1e12)
ax.set_ylim(-1e12, 1e12)
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title('Trayectoria en el eje x entre estrellas binarias')
ax.grid(True)

# Dibujar estrellas
ax.plot(x1, y1, 'ro', markersize=15, label='Próxima Centauri')
ax.plot(x2, y2, 'bo', markersize=20, label='Alfa Centauri B')
ax.legend()

# Inicializar elementos de la animación
line, = ax.plot([], [], 'g-', lw=2, label='Trayectoria')
point, = ax.plot([], [], 'ko', markersize=8, label='Asteroide')
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

def init():
    line.set_data([], [])
    point.set_data([], [])
    time_text.set_text('')
    return line, point, time_text

def update(frame):
    line.set_data(sol.y[0][:frame+1], sol.y[1][:frame+1])
    point.set_data([sol.y[0][frame]], [sol.y[1][frame]])

    # Mostrar tiempo en días
    dias = t_eval[frame] / (24 * 3600)
    time_text.set_text(f'Tiempo: {dias:.1f} días')

    return line, point, time_text

# Crear animación reducida a 50 frames, para mayor fluidez
anim = FuncAnimation(
    fig,
    update,
    frames=len(t_eval),
    init_func=init,
    blit=True,
    interval=100  # 100 ms entre frames
)

plt.close()
HTML(anim.to_html5_video())

A partir de la trayectoria evidenciada, es claro que el modelo cumple con las leyes de gravitación universal. El asteroide presenta una trayectoria aproximadamente elíptica alrededor de Alfa Centauri B, ya que esta tiene una masa mucho mayor que Próxima Centauri. Esto se corresponde con los modelos gravitatorios encontrados típicamente en el universo.

In [24]:
t_span = (0, 40e7)
t_eval = np.linspace(0, 40e7, 150)  # 150 frames
estado_inicial = [-1.75e12, 1.0e12, 1.5e3, -2.5e3]  # (x0, y0>0, vx0, vy0)

sol = solve_ivp(ecuaciones, t_span, estado_inicial, t_eval=t_eval, method='RK45')

fig, ax = plt.subplots(figsize=(6, 4))
ax.set_xlim(-2.5e12, 0.5e12)
ax.set_ylim(-1.5e12, 1.5e12)
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title('Trayectoria con altura inicial (y₀ > 0)')
ax.grid(True)

ax.plot(x1, y1, 'ro', markersize=18, label='Próxima Centauri')
ax.plot(x2, y2, 'bo', markersize=24, label='Alfa Centauri B')
ax.legend(loc='upper right')

line, = ax.plot([], [], 'm-', lw=2, alpha=0.7, label='Trayectoria')
point, = ax.plot([], [], 'ko', markersize=10, label='Asteroide')
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=12)

def init():
    line.set_data([], [])
    point.set_data([], [])
    time_text.set_text('')
    return line, point, time_text

def update(frame):
    x_data = sol.y[0][:frame+1]
    y_data = sol.y[1][:frame+1]
    line.set_data(x_data, y_data)

    point.set_data([sol.y[0][frame]], [sol.y[1][frame]])

    # Mostrar tiempo en años
    años = t_eval[frame] / (365.25 * 24 * 3600)
    time_text.set_text(f'Tiempo: {años:.2f} años')

    return line, point, time_text

anim = FuncAnimation(
    fig,
    update,
    frames=len(t_eval),
    init_func=init,
    blit=True,
    interval=80  # 80 ms entre frames
)

plt.close()
HTML(anim.to_html5_video())

La trayectoria mostrada evidencia que un cambio en las condiciones iniciales, poniendo al asteroide más cerca de Próxima Centauri y con una altura inicial $y_0 > 0$, cambia el modelo de trayectoria seguido por el cuerpo. El campo gravitacional de Próxima Centauri afecta la trayectoria al estar más cercano al asteroide, generando una órbita más similar a una parábola.

In [29]:
t_span = (0, 20e8)
t_eval = np.linspace(0, 20e8, 300)  # 300 frames
estado_inicial = [-3.0e12, 0, 0, -4.8e3]  # (x0, y0=0, vx0=0, vy0)

sol = solve_ivp(ecuaciones, t_span, estado_inicial, t_eval=t_eval, method='DOP853', rtol=1e-10)

fig, ax = plt.subplots(figsize=(5,5))
ax.set_xlim(-4e12, 2e12)
ax.set_ylim(-4e12, 2e12)
ax.set_aspect('equal')
ax.grid(True)
ax.set_title('Órbita casi circular alrededor del sistema binario', pad=20)

ax.plot(x1, y1, 'ro', markersize=12, label=f'Próxima Centauri ({M1/M2:.1f}M)')
ax.plot(x2, y2, 'bo', markersize=18, label=f'Alfa Centauri B (M)')
ax.legend(loc='upper right')

line, = ax.plot([], [], 'c-', lw=2, alpha=0.7, label='Trayectoria')
point, = ax.plot([], [], 'ko', markersize=10, label='Asteroide')
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=12,
                   bbox=dict(facecolor='white', alpha=0.8))

def init():
    line.set_data([], [])
    point.set_data([], [])
    time_text.set_text('')
    return line, point, time_text

def update(frame):
    start_frame = max(0, frame-100)
    line.set_data(sol.y[0][start_frame:frame+1], sol.y[1][start_frame:frame+1])
    point.set_data([sol.y[0][frame]], [sol.y[1][frame]])

    años = t_eval[frame] / (365.25 * 24 * 3600)
    time_text.set_text(f'Tiempo: {años:.2f} años\n'
                      f'Posición: ({sol.y[0][frame]/1e12:.2f}×10¹² m, {sol.y[1][frame]/1e12:.2f}×10¹² m)')

    return line, point, time_text

anim = FuncAnimation(
    fig,
    update,
    frames=len(t_eval),
    init_func=init,
    blit=True,
    interval=100
)

plt.close()
HTML(anim.to_html5_video())

#Conclusiones

Este proyecto ha demostrado cómo las simulaciones numéricas permiten estudiar trayectorias complejas en sistemas binarios de estrellas, revelando tanto su potencial como sus limitaciones. A través del modelo implementado, hemos observado que las condiciones iniciales—como la posición y velocidad del asteroide—juegan un papel crítico en su dinámica, generando comportamientos que van desde órbitas estables hasta trayectorias caóticas. Los errores en la integración numérica y la interpolación, aunque pequeños en escalas de tiempo cortas, se acumulan significativamente en simulaciones prolongadas, especialmente durante aproximaciones cercanas a las estrellas, donde las fuerzas gravitacionales son más intensas.  

La animación de las trayectorias no solo facilitó la visualización de estos fenómenos, sino que también destacó la sensibilidad del sistema a mínimas variaciones, un sello distintivo de los sistemas gravitacionales complejos. Aunque los métodos empleados—como Runge-Kutta y splines cúbicos—ofrecen resultados confiables para análisis cualitativos, el proyecto subraya la necesidad de técnicas más avanzadas para aplicaciones que requieran alta precisión, como misiones espaciales o estudios astrodinámicos detallados.  

En última instancia, el trabajo refuerza un principio clave en astrofísica computacional: la importancia de equilibrar precisión, eficiencia y claridad en las simulaciones. Herramientas como las animaciones interactivas y el análisis de errores no solo enriquecen la comprensión teórica, sino que también preparan el terreno para futuras investigaciones, donde podrían incorporarse elementos como perturbaciones de más cuerpos o efectos relativistas.